In [8]:
#vectype = "TFIDF"
#size = 50
#modelPath = os.path.join(cd, rf'{vectype}_{model}_{loadNumber}.pkl')
import tweepy as tp
import os
import pandas as pd
from joblib import load
user = (input("Enter your twitter handle here: ")).replace('@', '')
#put in choices for retweets and replies

In [9]:
def GetUserInput(username, max_results= 5, includingRetweets = False, includingReplies = True):
    bearer_token = "AAAAAAAAAAAAAAAAAAAAAAcblQEAAAAAtCqsQJwLtdMjU9olMpKT8mYnmjI%3D4707FKE1BToIY6TgtwgVzfNKAv7vohP1Uo3lDubgxwhUalWHi3"
    client_ID = "bjB6WHYzMlZhc3RYNF84Wlo0amM6MTpjaQ"
    client_secret = "-ZuUb0O6awVPBJLxGSVv7ezzt0PhxQjeMqaXh6JIJixiNKjGtl"
    redirect_uri = "https://troy.kli.ng/"
    consumer_key = "TUMFGbPsKwZ9UmOvJxjTNC2vo"
    consumer_secret = "pEzDJheRCGab7NG2phj5nnv7y6DovX4Ey9rfbHeqh7tv3t2ysY"
    access_token = "1615397853140242432-4gubCI7K70hryA84LHtAhmkXigRVAl"
    access_token_secret = "Z8JYzhlh5eiXkNy6q3oOY9SPEW7jFb62nLKuxf6FBmyGM"

    #authentication
    #I have all of these ready to go here, but only ended up needing 'bearer_token'.

    client = tp.Client(bearer_token=bearer_token)

    ID = client.get_user(username= username).data.id

    def add_tweet(tweetData):
        tweetList.append((tweetData.id, tweetData.text))

    def choose_exclusions(rt, rp):
        l = []
        if not rt:
            l.append('retweets')
        if not rp:
            l.append('replies')
        return l

    tweets = client.get_users_tweets(id=ID, max_results=max_results, exclude= choose_exclusions( \
        includingRetweets, includingReplies))

    tweetList = []

    for i in range(len(tweets.data)):
        add_tweet(tweets.data[i])
    
    return tweetList

In [10]:
print(GetUserInput('wtfsicckko'))

[(1665451497826537476, '@dayveedlol see this why i dont fw european niggas'), (1665451352812666881, '@tres_manxes thats real'), (1665451318889021442, '@joe3971 indecisive ass nigga'), (1665440070554222594, '@2drowsyy nasty 😷'), (1665414960107642880, 'sweaty sex or air conditioner sex')]


In [11]:
cd = r"C:\Users\marce\Project\models\TFIDF500" #whatever model folder i choose
modelPath = os.path.join(cd, 'Model_TFIDF_NaiveBayes_500.pkl')
vocabPath = os.path.join(cd, 'Vocab_500.pkl')
uInput = pd.DataFrame(GetUserInput(user), columns= ['ID', 'Text'])



In [12]:
#use whichever of these three I decide to joblib load
import sklearn.naive_bayes
import sklearn.linear_model
import sklearn.svm

In [13]:
print(uInput)

                    ID                                               Text
0  1665451497826537476  @dayveedlol see this why i dont fw european ni...
1  1665451352812666881                            @tres_manxes thats real
2  1665451318889021442                      @joe3971 indecisive ass nigga
3  1665440070554222594                                  @2drowsyy nasty 😷
4  1665414960107642880                  sweaty sex or air conditioner sex


In [19]:
from CleanTweetsScript import CleanDF, vectorize
cleaned, vocab = (CleanDF(uInput), load(vocabPath))
print(cleaned)
#cleaned.drop(labels='index', axis=1, inplace=True)
df, _ = vectorize(cleaned, vocab=vocab)
print(df)


   index                   ID  \
0      0  1665451497826537476   
1      0  1665451352812666881   
2      0  1665451318889021442   
3      0  1665440070554222594   
4      0  1665414960107642880   

                                                Text  \
0  @dayveedlol see this why i dont fw european ni...   
1                            @tres_manxes thats real   
2                      @joe3971 indecisive ass nigga   
3                                  @2drowsyy nasty 😷   
4                  sweaty sex or air conditioner sex   

                        Cleaned  
0         see fw european nigga  
1                          real  
2             indecis ass nigga  
3                         nasti  
4  sweati sex air condition sex  
                    ID                                               Text  \
0  1665451497826537476  @dayveedlol see this why i dont fw european ni...   
1  1665451352812666881                            @tres_manxes thats real   
2  1665451318889021442       

In [20]:
# 3:int((df.shape[1] - 3) / 2)

bow_df = df.iloc[:, 3:int((df.shape[1] - 3) / 2 + 3)]
bow_df = bow_df.reset_index(drop=True)

tfidf_df = df.iloc[:, int((df.shape[1] - 3) / 2 + 3):df.shape[1]]
tfidf_df = tfidf_df.reset_index(drop=True)

idNum = df.ID.reset_index(drop=True)

In [22]:
print(bow_df, tfidf_df, idNum)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4] Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4] 0    1665451497826537476
1    1665451352812666881
2    1665451318889021442
3    1665440070554222594
4    1665414960107642880
Name: ID, dtype: object


In [21]:


model = load(modelPath)
predictions = model.predict(tfidf_df)
predSeries = pd.Series(predictions)
fullDF = pd.concat(predSeries, df.iloc[:,[0,1]],axis= 1)
print(fullDF)

d:\Users\marce\miniconda3\envs\SiAI\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


ValueError: at least one array or dtype is required